In [ ]:
from pathflowai.utils import load_sql_df
import torch
import os 
import sys, os
sys.path.insert(0,os.path.abspath("dgm"))
import umap, numba
from sklearn.preprocessing import LabelEncoder
from torch_cluster import knn_graph
from torch_geometric.data import Data 
import numpy as np
from torch_geometric.utils import train_test_split_edges
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import argparse
from dgm.dgm import DGM
from dgm.plotting import *
from dgm.utils import *
from dgm.models import GraphClassifier, DGILearner
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
import os,glob, pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import pickle
import fire
import networkx as nx

In [ ]:
# load graph dataset and image
i,j=0,0 # cv_split, slide_idx in split
arr=cv2.resize(np.load("slideID.npy"),None,fx=1/6,fy=1/6,interpolation=cv2.INTER_CUBIC)
cv_splits=pickle.load(open("cv_splits/cv_splits.pkl",'rb'))[i]['test_idx']
pred=torch.load("predictions/predictions.pth")[-len(cv_splits):][j]
graph_dataset=pickle.load(open("graph_datasets/graph_dataset.pkl",'rb'))
X=pickle.load(open(f"design.matix.{i}",'rb'))

In [ ]:
import copy
def display_results(out_graph,res,predict=False,custom_colors=[],img=None,alpha=None,scatter=True,width_scale=20,node_scale=90):
    xy=pred["xy"]
    y_orig=pred[("y" if not predict else 'y_update')] 
    y=copy.deepcopy(y_orig)
    graph=out_graph
    binary=False
    node_color=res['mnode_to_color']; node_size=res['node_sizes']; edge_weight=res['edge_weight']
    if custom_colors: node_color=custom_colors
    node_list=res['node_list']; name='wsi'
    cmap = cm.coolwarm
    cmap = cm.get_cmap(cmap, 100)
    plt.set_cmap(cmap)

    f = plt.figure(figsize=(10,10))
    ax = f.add_subplot(1, 1, 1)

    edges = graph.edges()
    weights = np.array([edge_weight[(min(u, v), max(u, v))] for u, v in edges], dtype=np.float32)

    width = weights * width_scale

    node_size = np.sqrt(node_size) * node_scale
    c=y.flatten()

    scale=6
    pos = {}
    for node in graph.nodes():
        if len(res['mnode_to_nodes'][node])-1:
            pos[node]=np.array([xy[i] for i in res['mnode_to_nodes'][node]]).mean(0)/scale
        else:
            pos[node]=xy[list(res['mnode_to_nodes'][node])[0]]/scale
            
    if not isinstance(img,type(None)): plt.imshow(img)

    if scatter: plt.scatter(xy[:,0]/scale,xy[:,1]/scale,c=c,alpha=alpha)
    nx.draw(graph, pos=pos, node_color=node_color, width=width, node_size=node_size,
            node_list=node_list, ax=ax, cmap=cmap)
    return None

In [ ]:
dgm_res=X['dgm_res'][list(X['dgm_res'].keys())[j]]
display_results(dgm_res[0],dgm_res[1],custom_colors=node_colors,img=arr[::-1,:],alpha=0.1,scatter=False,node_scale=50)